# 2020.11.24 과제
### 내 주유소 지도위에 찍고 휘발유 가격 보여주기

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import requests
from urllib.parse import quote, unquote
import pandas as pd

In [3]:
key_fd = open('../06.RestAPI/roadapikey.txt', mode='r')
roadapi_key = key_fd.read(100)
key_fd.close()

In [8]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
my_home = '성복동 롯데캐슬골드타운'
url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote(my_home)}&resultType=json'
url

'https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey=U01TX0FVVEgyMDIwMTExOTEwMjMzNTExMDQzNzg=&currentPage=1&countPerPage=10&keyword=%EC%84%B1%EB%B3%B5%EB%8F%99%20%EB%A1%AF%EB%8D%B0%EC%BA%90%EC%8A%AC%EA%B3%A8%EB%93%9C%ED%83%80%EC%9A%B4&resultType=json'

In [10]:
result = requests.get(url).json()
my_addr = result['results']['juso'][1]['roadAddr']
my_addr

'경기도 용인시 수지구 성복2로 10 (성복동, 성복역롯데캐슬골드타운)'

In [23]:
df = pd.read_csv('주유소.csv')
df.head(5)

,광역,지역,상호,주소
0,경기,용인시수지구,서창산업㈜죽전(상)주유소,경기 용인시 수지구 용구대로2771번길 37 (죽전동)
1,경기,용인시수지구,풍덕천Self주유소,경기 용인시 수지구 포은대로 402 (풍덕천동)
2,경기,용인시수지구,대명에너지 광교셀프직영점,경기 용인시 수지구 포은대로 58 (상현동)
3,경기,용인시수지구,반도주유소,경기 용인시 수지구 신수로683번길 11 (풍덕천동)
4,경기,용인시수지구,지에스칼텍스㈜ 죽전주유소,경기 용인시 수지구 용구대로 2782 (죽전동)


In [28]:
driver = webdriver.Chrome('./chromedriver')
#driver.maximize_window()
driver.get('https://www.opinet.co.kr/user/main/mainView.do')
time.sleep(1)
driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)

addr_list = []; price_list = []
for i in df.index:
    driver.find_element_by_xpath('//*[@id="SIDO_NM0"]').send_keys(df['광역'][i])
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]').send_keys(df['지역'][i])
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="searRgSelect"]').click()
    time.sleep(8)
    driver.find_element_by_xpath('//*[@id="SEARCH_MOD4"]').click()
    time.sleep(8)
    station_name = driver.find_element_by_id('OS_NM')
    station_name.clear()
    station_name.send_keys(df['상호'][i])
    time.sleep(2)
    # 조회 버튼 클릭
    driver.find_element_by_id('searRgSelect').click()
    time.sleep(8)
    price = driver.find_element_by_css_selector('.price').text
    price_list.append(int(price))
    print(price)
    # 주유소명 Click
    """ driver.find_element_by_xpath('//*[@id="body1"]/tr/td[1]/a').click()
    time.sleep(10)
    popup = driver.find_element_by_css_selector('.ollehmap-info-defaultStyle')
    addr = popup.find_elements_by_tag_name('li')[1].text[7:]
    addr_list.append(addr)
    print(addr) """
    time.sleep(8)

1245
1298
1299


UnexpectedAlertPresentException: Alert Text: 시군구를 선택하여주세요
Message: unexpected alert open: {Alert text : 시군구를 선택하여주세요}
  (Session info: chrome=87.0.4280.66)


In [78]:
#df['addr'] = addr_list
df['price'] = price_list
df.head()

,name,addr,price
0,기흥주유소,경기 용인시 기흥구 공세로 173,1242
1,미키주유소,경기 용인시 기흥구 용구대로 2469번길 57,1255
2,구성그린주유소,경기 용인시 기흥구 용구대로 2421 (마북동),1265
3,보정꽃길주유소,경기 용인시 기흥구 용구대로2469번길 153,1265
4,이마트셀프전용주유소,경기 용인시 기흥구 용구대로 2441 (마북동),1268


In [79]:
df.sort_values('price', inplace=True)
df.to_csv('station.csv', sep=',', encoding='utf8', index=False)

### 위도, 경도 정보 구하기

In [25]:
key_fd = open('../06.RestAPI/kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [26]:
url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(my_addr)
result = requests.get(url,
            headers={"Authorization": "KakaoAK "+kmap_key}).json()
my_lat = float(result['documents'][0]['y'])
my_lng = float(result['documents'][0]['x'])
print(my_lat, my_lng)

37.3112644829202 127.08321416053


In [3]:
df = pd.read_csv('station.csv')
df

,name,addr,price
0,기흥주유소,경기 용인시 기흥구 공세로 173,1242
1,미키주유소,경기 용인시 기흥구 용구대로 2469번길 57,1255
2,구성그린주유소,경기 용인시 기흥구 용구대로 2421 (마북동),1265
3,보정꽃길주유소,경기 용인시 기흥구 용구대로2469번길 153,1265
4,이마트셀프전용주유소,경기 용인시 기흥구 용구대로 2441 (마북동),1268


In [4]:
lat_list = []; lng_list = []
for i in df.index:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(df.addr[i])
    result = requests.get(url,
            headers={"Authorization": "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [5]:
df['lat'] = lat_list
df['lng'] = lng_list
df

,name,addr,price,lat,lng
0,기흥주유소,경기 용인시 기흥구 공세로 173,1242,37.236377,127.104316
1,미키주유소,경기 용인시 기흥구 용구대로 2469번길 57,1255,37.311753,127.104677
2,구성그린주유소,경기 용인시 기흥구 용구대로 2421 (마북동),1265,37.301906,127.105711
3,보정꽃길주유소,경기 용인시 기흥구 용구대로2469번길 153,1265,37.320348,127.104484
4,이마트셀프전용주유소,경기 용인시 기흥구 용구대로 2441 (마북동),1268,37.303651,127.104802


In [6]:
df.to_csv('station2.csv', sep=',', encoding='utf8', index=False)

### 주유소 정보 지도위에 표시하기

In [7]:
import folium

df = pd.read_csv('station2.csv')

In [8]:
df.sort_values('price', inplace=True)

,name,addr,price,lat,lng
0,기흥주유소,경기 용인시 기흥구 공세로 173,1242,37.236377,127.104316
1,미키주유소,경기 용인시 기흥구 용구대로 2469번길 57,1255,37.311753,127.104677
2,구성그린주유소,경기 용인시 기흥구 용구대로 2421 (마북동),1265,37.301906,127.105711
3,보정꽃길주유소,경기 용인시 기흥구 용구대로2469번길 153,1265,37.320348,127.104484
4,이마트셀프전용주유소,경기 용인시 기흥구 용구대로 2441 (마북동),1268,37.303651,127.104802


In [12]:
df['color'] = ['red', 'blue', 'blue', 'blue', 'blue']
map = folium.Map(location=[df.lat.mean(), df.lng.mean()], zoom_start=13)
for i in df.index:
    folium.Marker(
        location=[df.lat[i], df.lng[i]],
        popup=df.name[i],
        tooltip=f'휘발유: {df.price[i]}원',
        icon=folium.Icon(color=df.color[i])
    ).add_to(map)
map